In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 098bbcdf-8f22-49c5-8fa0-81ce4d79f1bc, 3, Finished, Available, Finished)

In [3]:
%%sql 
create table if not EXISTS project_lakehouse.gold_product
(
    product_ID long ,
    product_Category string ,
    product string ,
    Create_TS timestamp,
    Modified_TS timestamp
)

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
Max_Date = spark.sql("""
    SELECT COALESCE(MAX(Modified_TS), '1900-01-01') 
    FROM project_lakehouse.gold_product
""").first()[0]

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 6, Finished, Available, Finished)

In [5]:
df_bronze = spark.sql("""
    SELECT product_Category, product
    FROM project_lakehouse.bronze_sales
    WHERE Modified_TS > '{}'
""".format(Max_Date))


StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 7, Finished, Available, Finished)

In [6]:
Max_ID=spark.sql("select coalesce(max( product_ID),0) from project_lakehouse.gold_product").first()[0]

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 8, Finished, Available, Finished)

In [7]:
df_final=df_bronze.withColumn("product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 9, Finished, Available, Finished)

In [8]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 10, Finished, Available, Finished)

In [9]:
%%sql
select * from viewProduct limit 10

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 11, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 3 fields>

In [11]:
%%sql

MERGE INTO project_lakehouse.gold_product AS gp
USING ViewProduct AS vp
ON gp.Product = vp.Product 
   AND gp.Product_Category = vp.Product_Category
   AND gp.Product_ID = vp.Product_ID
WHEN MATCHED THEN 
  UPDATE SET gp.Modified_TS = current_timestamp()
WHEN NOT MATCHED THEN 
  INSERT (Product_ID, Product_Category, Product, Create_TS, Modified_TS)
  VALUES (vp.Product_ID, vp.Product_Category, vp.Product, current_timestamp(), current_timestamp())


StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 13, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [13]:
%%sql 
select * from gold_product limit 10

StatementMeta(, 6a218c4d-6416-4b59-99ba-93ba372141bb, 15, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 5 fields>